In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import timedelta
import numpy as np

In [36]:
data = pd.read_csv("../data/data_ext2_2023-08-19.csv")
data = data.copy()

In [37]:
data.tail(10)

,group,sender,text,date
444148,berlinhelpsukrainians,1.542109e+09,@Yulia_uon,2023-08-19 09:00:15
444149,berlinhelpsukrainians,8.202858e+08,Имеет смысл,2023-08-19 09:01:59
444150,berlinhelpsukrainians,5.117098e+09,#furniture\n==============================\nОф...,2023-08-19 09:07:38
444151,berlinhelpsukrainians,4.652552e+08,Имеет.,2023-08-19 09:07:52
444152,berlinhelpsukrainians,8.330099e+08,"Да! Смысл есть! По глупости, я прислушалась к ...",2023-08-19 09:22:18
444153,berlinhelpsukrainians,2.686495e+08,Спасибо большое ☺️,2023-08-19 09:23:51
444154,berlinhelpsukrainians,5.024801e+09,"А мне совершенно не помогло, полгода попользов...",2023-08-19 09:25:35
444155,berlinhelpsukrainians,7.263066e+08,"Тогда скажите арендодателю, что вам не нужна э...",2023-08-19 09:28:44
444156,berlinhelpsukrainians,6.272974e+08,"Нет, одно другого не касается. Расходы на отоп...",2023-08-19 09:32:56
444157,berlinhelpsukrainians,6.272974e+08,Это можно спросить в отеле. У каждого свой набор,2023-08-19 09:34:33


## Cleaning

### Basic cleaning

In [51]:
len(data)
#444158

434772

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444158 entries, 0 to 444157
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   group   444158 non-null  object 
 1   sender  441655 non-null  float64
 2   text    434772 non-null  object 
 3   date    444158 non-null  object 
dtypes: float64(1), object(3)
memory usage: 13.6+ MB


In [40]:
data = data[data['text'].notna()]

In [41]:
data['text'] = data['text'].str.strip()

In [42]:
data['text'].replace('', np.nan, inplace=True)

In [43]:
data.dropna(subset=['text'], inplace=True)

### Merging rows together that belong to the same sender within a given timeframe

In [45]:
data['date'] = pd.to_datetime(data["date"])

In [60]:
df = data.sort_values(by=['sender', 'date'])

df['time_diff'] = df.groupby('sender')['date'].diff()

data_clean = df.groupby(['sender', (df['time_diff'] > timedelta(minutes=45)).cumsum()]).agg({'text': ' '.join, 
                                                                                         'date': 'first'}).reset_index()
data_clean

,sender,time_diff,text,date
0,-1.001789e+12,0,https://t.me/pomogatorua,2022-05-14 06:39:42
1,-1.001789e+12,1,https://t.me/dddkkkke,2022-05-15 20:28:00
2,-1.001782e+12,1,Можно ли получить внж если меня пригласит мой ...,2022-03-23 15:27:24
3,-1.001782e+12,2,а если я получу внж и потом сразу уезжаю в Укр...,2022-03-23 23:08:05
4,-1.001782e+12,3,в Берлине щас долгая регистрация? прописка и внж,2022-03-24 20:42:05
...,...,...,...,...
184351,6.517925e+09,159207,hello everyone,2023-08-04 08:01:41
184352,6.638994e+09,159207,"Добрый вечер, дамы и господа. Скажите пожалуйс...",2023-08-15 17:54:05
184353,6.638994e+09,159208,"Спасибо за ответ, решил не ждать написал письм...",2023-08-15 20:44:20
184354,6.638994e+09,159209,"Добрый день ребята, у меня вопрос я одно время...",2023-08-18 12:28:33


In [61]:
len(data_clean)
# 45 mins > 184356
# 30 mins > 196122
# 15 mins > 219198
# 10 mins > 235163
# 5 mins > 271890

184356

### DateTime index

In [ ]:
data['date'] = pd.to_datetime(data["date"])
data = data.set_index("date")

In [ ]:
##